<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/440_PDO_ReportGen_Utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# 1. Architectural Quality: This Is the Right Reporting Layer

This is not “string concatenation dressed up as a report.”

This is a **structured reporting pipeline** with clear responsibilities:

| Layer              | Responsibility           |
| ------------------ | ------------------------ |
| Format helpers     | Presentation consistency |
| Section generators | Business-domain logic    |
| Report composer    | Assembly + flow          |
| Persistence        | Side-effect isolation    |

That separation is *exactly* what keeps report logic sane as the system grows.

### Strong design signals

* Every section is independently testable
* No section mutates state
* No hidden dependencies
* No LLM assumptions
* Clean fallback behavior everywhere

This is **production-grade**.

---

# 2. Executive Readiness: This Will Land Well With CEOs

Your executive summary is doing the *right job*:

### What it does well

* Leads with **mode + scope**
* Shows **ROI first**
* Shows **workflow health second**
* Shows **KPI posture third**
* Uses icons sparingly and meaningfully
* Avoids technical language unless necessary

This is *exactly* the reading order executives expect.

> Most reports bury ROI halfway down the page.
> Yours leads with it.

That alone puts this above 90% of internal dashboards.

---

# 3. KPI Section: Clean, Scannable, Trustworthy

You made several smart choices here:

### a. Status icons are consistent across KPI classes

Operational, Effectiveness, and Business KPIs all follow the same pattern:

* Value
* Status
* Visual indicator

That consistency builds **cognitive trust**.

### b. Formatting logic is centralized

`format_kpi_value`, `format_currency`, `format_percentage` are doing real work here.

That means:

* No random formatting drift
* Easy global changes later
* Safe LLM handoff later (if needed)

---

# 4. ROI Section: Transparent, Not Salesy

This is *very* well done.

You:

* Show cost
* Show revenue
* Show net ROI
* Show ratio
* Show efficiency assessment

And you **don’t hide negative ROI**.

That’s the difference between:

> “AI demo”
> and
> “Decision-support system”

Executives *trust* systems that admit when things aren’t working.

---

# 5. Workflow Analysis: Diagnosable, Not Overwhelming

This section strikes a great balance:

### What you got right

* Health status is clear
* Metrics are interpretable
* Bottlenecks are ranked
* Stage table is capped (top 10)
* Language is business-readable

You avoided the trap of dumping raw telemetry.

---

# 6. Recommendations Section: Actionable Without Being Prescriptive

This is one of the strongest parts of the entire agent.

Why?

Because your recommendations are:

* Conditional
* Evidence-driven
* Prioritized
* Limited in number

You are **not** pretending the system knows what to do.

You are saying:

> “Based on what we measured, here’s what deserves attention.”

That aligns *perfectly* with your core principle:

> *“The LLM does not decide what the business should do — it explains what the system has already proven.”*

Even without an LLM here, that philosophy is baked in.

---

# 7. Executive Report Assembly: Clean, Predictable Flow

Your `generate_executive_report()` function is excellent.

### Highlights

* Clear section boundaries
* Conditional inclusion
* No missing-section crashes
* Errors surfaced at the end (not hidden)
* Clean footer attribution

This will render beautifully in:

* Markdown viewers
* GitHub
* Notion
* Email previews
* PDF export later

---

# 8. Saving Reports: Correct Side-Effect Isolation

Your `save_executive_report()` function is doing exactly what it should:

* No formatting logic
* No business logic
* Just naming + persistence

That’s textbook clean architecture.

---

# 9. Minor Refinements (Optional, Not Required)

These are **enhancements**, not fixes.

### 1. “What would change my mind?” section (future)

You already discussed this concept earlier.

This report is now *ready* for a configurable section like:

```markdown
## Monitoring Thresholds

The following conditions would trigger concern:
- ROI ratio < 1.5
- Workflow health = critical
- >20% KPIs at risk
```

Drop-in ready later.

---

### 2. Report metadata block (optional)

Some orgs like a small metadata header:

```markdown
**Agent Version:** v1.0  
**Run ID:** 2026-01-13-001  
**Data Snapshot:** 2026-01-12
```

Not necessary now, but your structure supports it cleanly.

---

### 3. Snapshot tests later

Eventually, you may want golden markdown snapshots to prevent regressions.

Not needed for MVP.

---

# 10. Strategic Verdict

This report layer proves something important:

> You are not building “agents that talk.”
> You are building **systems that explain outcomes**.

At this point, you have:

* A fully deterministic agent
* Measured KPIs
* Transparent ROI
* Diagnosable workflow health
* Executive-ready reporting
* Passing end-to-end tests

You are **absolutely ready** to add:

* LLM summarization (optional)
* Natural-language Q&A over reports
* Multi-agent portfolio rollups
* Cross-agent executive dashboards

But you don’t *need* any of that to deliver value.


In [ ]:
"""Report Generation Utilities for Proposal & Document Orchestrator

These utilities generate executive markdown reports with business impact,
KPIs, ROI, workflow analysis, and recommendations.

Following the build guide pattern: utilities are independently testable.
"""

from typing import Dict, Any, List, Optional
from datetime import datetime
from toolshed.reporting.file_handling import save_report


def format_kpi_value(value: Any) -> str:
    """Format KPI value for display"""
    if isinstance(value, float):
        if value >= 1000:
            return f"{value:,.2f}"
        elif value >= 1:
            return f"{value:.2f}"
        else:
            return f"{value:.3f}"
    elif isinstance(value, int):
        return f"{value:,}"
    else:
        return str(value)


def format_percentage(value: float) -> str:
    """Format percentage value"""
    return f"{value:.1f}%"


def format_currency(value: float) -> str:
    """Format currency value"""
    return f"${value:,.2f}"


def generate_executive_summary(
    state: Dict[str, Any],
    goal: Dict[str, Any]
) -> str:
    """
    Generate executive summary section.

    Args:
        state: Complete orchestrator state
        goal: Goal definition

    Returns:
        Markdown string for executive summary
    """
    analysis_mode = goal.get("analysis_mode", "portfolio")
    document_id = state.get("document_id")

    # Get key metrics
    total_cost = state.get("total_cost_usd", 0.0)
    net_roi = state.get("net_roi_usd", 0.0)
    roi_status = state.get("roi_status", "neutral")
    roi_percent = state.get("roi_percent", 0.0)

    # Get workflow health
    workflow_analysis = state.get("workflow_analysis", {})
    workflow_health = workflow_analysis.get("workflow_health", {})
    health_status = workflow_health.get("workflow_health", "unknown")

    # Get KPI status
    kpi_status = state.get("kpi_status", {})

    # Count documents analyzed
    document_analysis = state.get("document_analysis", [])
    num_documents = len(document_analysis) if analysis_mode == "portfolio" else 1

    summary = f"""## Executive Summary

**Analysis Mode:** {analysis_mode.title()}
"""

    if analysis_mode == "single":
        summary += f"**Document ID:** {document_id}\n"
    else:
        summary += f"**Documents Analyzed:** {num_documents}\n"

    summary += f"**Generated:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n"

    # ROI Summary
    roi_icon = "✅" if roi_status == "positive" else "⚠️" if roi_status == "negative" else "➖"
    summary += f"### ROI Analysis\n\n"
    summary += f"{roi_icon} **Net ROI:** {format_currency(net_roi)} ({format_percentage(roi_percent)})\n"
    summary += f"**Total Cost:** {format_currency(total_cost)}\n"
    summary += f"**ROI Status:** {roi_status.title()}\n\n"

    # Workflow Health
    health_icon = "✅" if health_status == "healthy" else "⚠️" if health_status == "degraded" else "❌"
    summary += f"### Workflow Health\n\n"
    summary += f"{health_icon} **Status:** {health_status.title()}\n"

    if health_status != "healthy":
        health_issues = workflow_health.get("health_issues", [])
        if health_issues:
            summary += f"**Issues:** {len(health_issues)} identified\n"

    summary += "\n"

    # KPI Status Summary
    if kpi_status:
        on_track = sum(1 for s in kpi_status.values() if s == "on_track")
        at_risk = sum(1 for s in kpi_status.values() if s == "at_risk")
        exceeded = sum(1 for s in kpi_status.values() if s == "exceeded")

        summary += f"### KPI Status\n\n"
        summary += f"✅ **On Track:** {on_track}\n"
        summary += f"⚠️ **At Risk:** {at_risk}\n"
        summary += f"🎯 **Exceeded:** {exceeded}\n\n"

    return summary


def generate_kpi_section(
    operational_kpis: Dict[str, Any],
    effectiveness_kpis: Dict[str, Any],
    business_kpis: Dict[str, Any],
    kpi_status: Dict[str, str]
) -> str:
    """
    Generate KPI metrics section.

    Args:
        operational_kpis: Operational KPI metrics
        effectiveness_kpis: Effectiveness KPI metrics
        business_kpis: Business KPI metrics
        kpi_status: KPI status dictionary

    Returns:
        Markdown string for KPI section
    """
    section = "## KPI Metrics\n\n"

    # Operational KPIs
    if operational_kpis:
        section += "### Operational KPIs (Agent Health)\n\n"
        for key, value in operational_kpis.items():
            status = kpi_status.get(key, "unknown")
            status_icon = "✅" if status == "on_track" else "⚠️" if status == "at_risk" else "🎯" if status == "exceeded" else "➖"
            formatted_key = key.replace("_", " ").title()
            formatted_value = format_kpi_value(value)

            # Add percentage sign for rates
            if "rate" in key or "percent" in key:
                formatted_value = format_percentage(value * 100) if isinstance(value, float) and value < 1 else formatted_value

            section += f"{status_icon} **{formatted_key}:** {formatted_value}\n"
        section += "\n"

    # Effectiveness KPIs
    if effectiveness_kpis:
        section += "### Effectiveness KPIs (Workflow Quality)\n\n"
        for key, value in effectiveness_kpis.items():
            status = kpi_status.get(key, "unknown")
            status_icon = "✅" if status == "on_track" else "⚠️" if status == "at_risk" else "🎯" if status == "exceeded" else "➖"
            formatted_key = key.replace("_", " ").title()
            formatted_value = format_kpi_value(value)

            # Add percentage sign for rates
            if "reduction" in key or "percent" in key:
                formatted_value = format_percentage(value * 100) if isinstance(value, float) and value < 1 else formatted_value

            section += f"{status_icon} **{formatted_key}:** {formatted_value}\n"
        section += "\n"

    # Business KPIs
    if business_kpis:
        section += "### Business KPIs (ROI & Value)\n\n"
        for key, value in business_kpis.items():
            status = kpi_status.get(key, "unknown")
            status_icon = "✅" if status == "on_track" else "⚠️" if status == "at_risk" else "🎯" if status == "exceeded" else "➖"
            formatted_key = key.replace("_", " ").title()
            formatted_value = format_kpi_value(value)

            # Format currency
            if "cost" in key or "revenue" in key or "roi" in key:
                if isinstance(value, (int, float)):
                    formatted_value = format_currency(value)

            # Add percentage sign for rates
            if "reduction" in key or "percent" in key:
                formatted_value = format_percentage(value * 100) if isinstance(value, float) and value < 1 else formatted_value

            section += f"{status_icon} **{formatted_key}:** {formatted_value}\n"
        section += "\n"

    return section


def generate_roi_section(
    total_cost_usd: float,
    total_revenue_impact_usd: float,
    net_roi_usd: float,
    roi_percent: float,
    roi_ratio: float,
    roi_status: str,
    cost_efficiency: Dict[str, Any]
) -> str:
    """
    Generate ROI analysis section.

    Args:
        total_cost_usd: Total cost
        total_revenue_impact_usd: Total revenue impact
        net_roi_usd: Net ROI
        roi_percent: ROI percentage
        roi_ratio: ROI ratio
        roi_status: ROI status
        cost_efficiency: Cost efficiency assessment

    Returns:
        Markdown string for ROI section
    """
    section = "## ROI & Cost Analysis\n\n"

    roi_icon = "✅" if roi_status == "positive" else "⚠️" if roi_status == "negative" else "➖"

    section += f"{roi_icon} **ROI Status:** {roi_status.title()}\n\n"

    section += "### Financial Summary\n\n"
    section += f"**Total Cost:** {format_currency(total_cost_usd)}\n"
    section += f"**Total Revenue Impact:** {format_currency(total_revenue_impact_usd)}\n"
    section += f"**Net ROI:** {format_currency(net_roi_usd)}\n"
    section += f"**ROI Percentage:** {format_percentage(roi_percent)}\n"
    section += f"**ROI Ratio:** {format_kpi_value(roi_ratio)}\n\n"

    # Cost Efficiency
    if cost_efficiency:
        section += "### Cost Efficiency Assessment\n\n"
        efficiency_status = cost_efficiency.get("status", "unknown")
        efficiency_icon = "✅" if efficiency_status == "efficient" else "⚠️"
        section += f"{efficiency_icon} **Status:** {efficiency_status.title()}\n"

        if "message" in cost_efficiency:
            section += f"**Assessment:** {cost_efficiency['message']}\n"
        section += "\n"

    return section


def generate_workflow_analysis_section(
    workflow_analysis: Dict[str, Any]
) -> str:
    """
    Generate workflow analysis section.

    Args:
        workflow_analysis: Workflow analysis results

    Returns:
        Markdown string for workflow analysis section
    """
    section = "## Workflow Analysis\n\n"

    workflow_health = workflow_analysis.get("workflow_health", {})
    health_status = workflow_health.get("workflow_health", "unknown")
    health_icon = "✅" if health_status == "healthy" else "⚠️" if health_status == "degraded" else "❌"

    section += f"{health_icon} **Overall Health:** {health_status.title()}\n\n"

    # Health Metrics
    section += "### Health Metrics\n\n"
    section += f"**Success Rate:** {format_percentage(workflow_health.get('overall_success_rate', 0.0) * 100)}\n"
    section += f"**Failure Rate:** {format_percentage(workflow_health.get('overall_failure_rate', 0.0) * 100)}\n"
    section += f"**Avg Stage Duration:** {format_kpi_value(workflow_health.get('avg_stage_duration_minutes', 0.0))} minutes\n"
    section += f"**Requires Attention:** {'Yes' if workflow_health.get('requires_attention', False) else 'No'}\n\n"

    # Health Issues
    health_issues = workflow_health.get("health_issues", [])
    if health_issues:
        section += "### Health Issues\n\n"
        for issue in health_issues:
            section += f"⚠️ {issue}\n"
        section += "\n"

    # Bottleneck Stages
    bottleneck_stages = workflow_analysis.get("bottleneck_stages", [])
    if bottleneck_stages:
        section += "### Bottleneck Stages\n\n"
        for bottleneck in bottleneck_stages[:5]:  # Top 5
            stage_name = bottleneck.get("stage_name", "unknown")
            duration = bottleneck.get("avg_duration_minutes", 0.0)
            failure_rate = bottleneck.get("failure_rate", 0.0)
            reason = bottleneck.get("bottleneck_reason", "unknown")

            section += f"**{stage_name.replace('_', ' ').title()}**\n"
            section += f"- Avg Duration: {format_kpi_value(duration)} minutes\n"
            section += f"- Failure Rate: {format_percentage(failure_rate * 100)}\n"
            section += f"- Reason: {reason.replace('_', ' ').title()}\n\n"

    # Stage Performance Summary
    stage_performance = workflow_analysis.get("stage_performance", {})
    if stage_performance:
        section += "### Stage Performance Summary\n\n"
        section += "| Stage | Executions | Success Rate | Avg Duration |\n"
        section += "|-------|------------|--------------|--------------|\n"

        for stage_name, perf in list(stage_performance.items())[:10]:  # Top 10
            executions = perf.get("total_executions", 0)
            success_rate = perf.get("success_rate", 0.0)
            avg_duration = perf.get("avg_duration_minutes", 0.0)

            section += f"| {stage_name.replace('_', ' ').title()} | {executions} | {format_percentage(success_rate * 100)} | {format_kpi_value(avg_duration)} min |\n"
        section += "\n"

    return section


def generate_portfolio_summary_section(
    portfolio_summary: Dict[str, Any],
    analysis_mode: str
) -> str:
    """
    Generate portfolio summary section.

    Args:
        portfolio_summary: Portfolio summary results
        analysis_mode: Analysis mode ("single" or "portfolio")

    Returns:
        Markdown string for portfolio summary section
    """
    if analysis_mode == "single":
        return ""  # Skip portfolio summary for single document analysis

    section = "## Portfolio Summary\n\n"

    section += f"**Total Documents:** {portfolio_summary.get('total_documents', 0)}\n"
    section += f"**Total Versions:** {portfolio_summary.get('total_versions', 0)}\n"
    section += f"**Total Reviews:** {portfolio_summary.get('total_reviews', 0)}\n"
    section += f"**Total Compliance Checks:** {portfolio_summary.get('total_compliance_checks', 0)}\n\n"

    # Documents by Type
    documents_by_type = portfolio_summary.get("documents_by_type", {})
    if documents_by_type:
        section += "### Documents by Type\n\n"
        for doc_type, count in documents_by_type.items():
            section += f"- **{doc_type.replace('_', ' ').title()}:** {count}\n"
        section += "\n"

    # Documents by Status
    documents_by_status = portfolio_summary.get("documents_by_status", {})
    if documents_by_status:
        section += "### Documents by Status\n\n"
        for status, count in documents_by_status.items():
            section += f"- **{status.replace('_', ' ').title()}:** {count}\n"
        section += "\n"

    # Documents by Priority
    documents_by_priority = portfolio_summary.get("documents_by_priority", {})
    if documents_by_priority:
        section += "### Documents by Priority\n\n"
        for priority, count in documents_by_priority.items():
            section += f"- **{priority.replace('_', ' ').title()}:** {count}\n"
        section += "\n"

    return section


def generate_recommendations_section(
    workflow_analysis: Dict[str, Any],
    kpi_status: Dict[str, str],
    roi_status: str
) -> str:
    """
    Generate recommendations section.

    Args:
        workflow_analysis: Workflow analysis results
        kpi_status: KPI status dictionary
        roi_status: ROI status

    Returns:
        Markdown string for recommendations section
    """
    section = "## Recommendations\n\n"

    recommendations = []

    # ROI Recommendations
    if roi_status == "negative":
        recommendations.append({
            "priority": "High",
            "title": "Improve ROI",
            "description": "Current ROI is negative. Focus on reducing costs or increasing revenue impact per hour saved."
        })

    # Workflow Health Recommendations
    workflow_health = workflow_analysis.get("workflow_health", {})
    health_status = workflow_health.get("workflow_health", "unknown")

    if health_status != "healthy":
        recommendations.append({
            "priority": "High" if health_status == "critical" else "Medium",
            "title": "Address Workflow Health Issues",
            "description": f"Workflow health is {health_status}. Review and address identified health issues."
        })

    # Bottleneck Recommendations
    bottleneck_stages = workflow_analysis.get("bottleneck_stages", [])
    if bottleneck_stages:
        top_bottleneck = bottleneck_stages[0]
        recommendations.append({
            "priority": "Medium",
            "title": f"Optimize {top_bottleneck.get('stage_name', 'Stage').replace('_', ' ').title()}",
            "description": f"This stage has high duration ({top_bottleneck.get('avg_duration_minutes', 0):.1f} min) and/or failure rate ({top_bottleneck.get('failure_rate', 0):.1%})."
        })

    # KPI Recommendations
    at_risk_kpis = [k for k, v in kpi_status.items() if v == "at_risk"]
    if at_risk_kpis:
        recommendations.append({
            "priority": "Medium",
            "title": "Address At-Risk KPIs",
            "description": f"The following KPIs are at risk: {', '.join([k.replace('_', ' ').title() for k in at_risk_kpis[:3]])}."
        })

    # Default recommendation if none
    if not recommendations:
        recommendations.append({
            "priority": "Low",
            "title": "Continue Monitoring",
            "description": "All systems are operating within acceptable parameters. Continue monitoring KPIs and workflow health."
        })

    # Format recommendations
    for i, rec in enumerate(recommendations, 1):
        priority_icon = "🔴" if rec["priority"] == "High" else "🟡" if rec["priority"] == "Medium" else "🟢"
        section += f"{i}. {priority_icon} **{rec['title']}** (Priority: {rec['priority']})\n"
        section += f"   {rec['description']}\n\n"

    return section


def generate_executive_report(state: Dict[str, Any]) -> str:
    """
    Generate complete executive markdown report.

    Args:
        state: Complete orchestrator state

    Returns:
        Complete markdown report string
    """
    goal = state.get("goal", {})
    analysis_mode = goal.get("analysis_mode", "portfolio")

    # Build report
    report = f"""# Proposal & Document Orchestrator - Executive Report

"""

    # Executive Summary
    report += generate_executive_summary(state, goal)
    report += "\n---\n\n"

    # KPI Metrics
    operational_kpis = state.get("operational_kpis", {})
    effectiveness_kpis = state.get("effectiveness_kpis", {})
    business_kpis = state.get("business_kpis", {})
    kpi_status = state.get("kpi_status", {})

    if operational_kpis or effectiveness_kpis or business_kpis:
        report += generate_kpi_section(
            operational_kpis,
            effectiveness_kpis,
            business_kpis,
            kpi_status
        )
        report += "---\n\n"

    # ROI Analysis
    total_cost = state.get("total_cost_usd", 0.0)
    total_revenue_impact = state.get("total_revenue_impact_usd", 0.0)
    net_roi = state.get("net_roi_usd", 0.0)
    roi_percent = state.get("roi_percent", 0.0)
    roi_ratio = state.get("roi_ratio", 0.0)
    roi_status = state.get("roi_status", "neutral")
    cost_efficiency = state.get("cost_efficiency", {})

    report += generate_roi_section(
        total_cost,
        total_revenue_impact,
        net_roi,
        roi_percent,
        roi_ratio,
        roi_status,
        cost_efficiency
    )
    report += "---\n\n"

    # Workflow Analysis
    workflow_analysis = state.get("workflow_analysis", {})
    if workflow_analysis:
        report += generate_workflow_analysis_section(workflow_analysis)
        report += "---\n\n"

    # Portfolio Summary
    portfolio_summary = state.get("portfolio_summary", {})
    if portfolio_summary:
        portfolio_section = generate_portfolio_summary_section(portfolio_summary, analysis_mode)
        if portfolio_section:
            report += portfolio_section
            report += "---\n\n"

    # Recommendations
    report += generate_recommendations_section(
        workflow_analysis,
        kpi_status,
        roi_status
    )
    report += "---\n\n"

    # Errors
    errors = state.get("errors", [])
    if errors:
        report += "## Errors\n\n"
        for error in errors:
            report += f"⚠️ {error}\n"
        report += "\n---\n\n"

    # Footer
    report += f"*Report generated by Proposal & Document Orchestrator Agent*\n"
    report += f"*Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}*\n"

    return report


def save_executive_report(
    report_content: str,
    state: Dict[str, Any],
    config: Any,
    report_id: Optional[str] = None
) -> str:
    """
    Save executive report to file.

    Args:
        report_content: Report markdown content
        state: Orchestrator state
        config: Agent configuration
        report_id: Optional report ID (defaults to document_id or timestamp)

    Returns:
        Path to saved report file
    """
    # Generate report ID
    if not report_id:
        document_id = state.get("document_id")
        if document_id:
            report_id = f"doc_{document_id}"
        else:
            report_id = "portfolio"

    # Use reports_dir from config
    reports_dir = getattr(config, "reports_dir", "output/proposal_document_orchestrator_reports")

    # Save report
    filepath = save_report(
        report_content=report_content,
        report_id=report_id,
        reports_dir=reports_dir,
        prefix="proposal_document_orchestrator"
    )

    return filepath
